In [1]:
import nltk
import random
from nltk.tokenize import word_tokenize
from collections import defaultdict
import math

corpus = "This is a sample corpus used for demonstration purposes. You can replace it with your own corpus data."


In [2]:
# Split the corpus into training, validation, and test sets
def split_corpus(corpus):
    total_length = len(corpus)
    train_end = int(total_length * 0.7)
    val_end = int(total_length * 0.8)
    
    train_set = corpus[:train_end]
    val_set = corpus[train_end:val_end]
    test_set = corpus[val_end:]
    
    return train_set, val_set, test_set

# Tokenize the text and limit the vocabulary size
def tokenize_text(text, vocab_size):
    tokens = word_tokenize(text.lower())
    freq_dist = nltk.FreqDist(tokens)
    vocab = [token for token, _ in freq_dist.most_common(vocab_size)]
    
    tokenized_text = []
    for token in tokens:
        if token in vocab:
            tokenized_text.append(token)
        else:
            tokenized_text.append('<UNK>')
    
    return tokenized_text

# Build 4-gram language model using backoff method
def build_lm_backoff(tokenized_text):
    unigram_counts = defaultdict(int)
    bigram_counts = defaultdict(int)
    trigram_counts = defaultdict(int)
    quadgram_counts = defaultdict(int)

    for i in range(len(tokenized_text)):
        token = tokenized_text[i]
        unigram_counts[token] += 1
        if i > 0:
            bigram = (tokenized_text[i-1], token)
            bigram_counts[bigram] += 1
        if i > 1:
            trigram = (tokenized_text[i-2], tokenized_text[i-1], token)
            trigram_counts[trigram] += 1
        if i > 2:
            quadgram = (tokenized_text[i-3], tokenized_text[i-2], tokenized_text[i-1], token)
            quadgram_counts[quadgram] += 1

    def compute_backoff_quadgram(quadgram):
        if quadgram_counts[quadgram] > 0:
            return quadgram_counts[quadgram] / trigram_counts[quadgram[:3]]
        else:
            return 0.4 * compute_backoff_trigram(quadgram[1:])

    def compute_backoff_trigram(trigram):
        if trigram_counts[trigram] > 0:
            return trigram_counts[trigram] / bigram_counts[trigram[:2]]
        else:
            return 0.4 * compute_backoff_bigram(trigram[1:])

    def compute_backoff_bigram(bigram):
        if bigram_counts[bigram] > 0:
            return bigram_counts[bigram] / unigram_counts[bigram[0]]
        else:
            return 0.4 * unigram_counts[bigram[0]] / len(tokenized_text)

    def compute_backoff_unigram(unigram):
        return unigram_counts[unigram] / len(tokenized_text)

    return compute_backoff_quadgram



In [3]:
# Build 4-gram language model using interpolation method with add-k smoothing
def build_lm_interpolation(tokenized_text, k, s):
    unigram_counts = defaultdict(int)
    bigram_counts = defaultdict(int)
    trigram_counts = defaultdict(int)
    quadgram_counts = defaultdict(int)

    for i in range(len(tokenized_text)):
        token = tokenized_text[i]
        unigram_counts[token] += 1
        if i > 0:
            bigram = (tokenized_text[i-1], token)
            bigram_counts[bigram] += 1
        if i > 1:
            trigram = (tokenized_text[i-2], tokenized_text[i-1], token)
            trigram_counts[trigram] += 1
        if i > 2:
            quadgram = (tokenized_text[i-3], tokenized_text[i-2], tokenized_text[i-1], token)
            quadgram_counts[quadgram] += 1

    V = len(unigram_counts)
    N = len(tokenized_text)

    def compute_prob_quadgram(quadgram):
        if quadgram_counts[quadgram] > 0:
            return (quadgram_counts[quadgram] + k) / (trigram_counts[quadgram[:3]] + k * V)
        else:
            return (s * compute_prob_trigram(quadgram[1:]) +
                    (1 - s) * compute_prob_trigram(quadgram[1:3]) +
                    (1 - s) * compute_prob_bigram(quadgram[2:]) +
                    (1 - s) * compute_prob_unigram(quadgram[3:]))

    def compute_prob_trigram(trigram):
        if trigram_counts[trigram] > 0:
            return (trigram_counts[trigram] + k) / (bigram_counts[trigram[:2]] + k * V)
        else:
            return (s * compute_prob_bigram(trigram[1:]) +
                    (1 - s) * compute_prob_bigram(trigram[1:2]) +
                    (1 - s) * compute_prob_unigram(trigram[2:]))

    def compute_prob_bigram(bigram):
        if bigram_counts[bigram] > 0:
            return (bigram_counts[bigram] + k) / (unigram_counts[bigram[0]] + k * V)
        else:
            return (s * compute_prob_unigram(bigram[1:]) +
                    (1 - s) * compute_prob_unigram(bigram[1:2]))

    def compute_prob_unigram(unigram):
        return (unigram_counts[unigram] + k) / (N + k * V)

    return compute_prob_quadgram


In [4]:
# Evaluate perplexity of a language model on a test set
def evaluate_perplexity(test_set, lm):
    tokenized_test_set = word_tokenize(test_set.lower())
    N = len(tokenized_test_set)
    log_prob_sum = 0

    for i in range(len(tokenized_test_set) - 3):
        quadgram = (tokenized_test_set[i], tokenized_test_set[i+1], tokenized_test_set[i+2], tokenized_test_set[i+3])
        prob = lm(quadgram)
        log_prob_sum += math.log2(prob) if prob > 0 else float('-inf')

    perplexity = 2 ** (-1/N * log_prob_sum)
    return perplexity


In [5]:
# Generate text using the language model
def generate_text(lm, max_length=100):
    generated_text = []
    start_tokens = ("<s>","<s>","<s>")
    for i in range(max_length):
        next_word = generate_next_word(lm, start_tokens)
        next_word = generate_next_word(lm, start_tokens)
        if next_word == "</s>":
            break
        generated_text.append(next_word)
        start_tokens = (start_tokens[1], start_tokens[2], next_word)

    return ' '.join(generated_text)
    
def generate_next_word(lm, tokens):
    candidates = []
    for token in set(tokens):
        quadgram = tokens + (token,)
        candidates.append((token, lm(quadgram)))
    sorted_candidates = sorted(candidates, key=lambda x: x[1], reverse=True)
    return sorted_candidates[0][0]

In [6]:
# Sample usage
train_set, val_set, test_set = split_corpus(corpus)

# Tokenize the training set
vocab_size = 10  # Adjust this according to your preference
tokenized_train_set = tokenize_text(train_set, vocab_size)

# Build language models
lm_backoff = build_lm_backoff(tokenized_train_set)
k = 0.5  # Add-k smoothing parameter
s = 0.5  # Interpolation parameter
lm_interpolation = build_lm_interpolation(tokenized_train_set, k, s)

# Evaluate perplexity
perplexity_backoff = evaluate_perplexity(test_set, lm_backoff)
perplexity_interpolation = evaluate_perplexity(test_set, lm_interpolation)

print("Perplexity for LM with backoff method:", perplexity_backoff)
print("Perplexity for LM with interpolation method:", perplexity_interpolation)

# Generate text using both models
generated_text_backoff = generate_text(lm_backoff)
generated_text_interpolation = generate_text(lm_interpolation)

print("\nGenerated text with backoff method:")
print(generated_text_backoff)

print("\nGenerated text with interpolation method:")
print(generated_text_interpolation)


Perplexity for LM with backoff method: inf
Perplexity for LM with interpolation method: 2.9383578541984376

Generated text with backoff method:
<s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s>

Generated text with interpolation method:
<s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s> <s>
